In [2]:
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
#jupyter内で画像を表示
%matplotlib inline 
#別ウィンドウで画像表示
# %matplotlib qt 
import csv
import json
from ipywidgets import interact, interactive, fixed, RadioButtons
import ipywidgets as widgets
from IPython.display import display
import pathlib
import glob
import re
import copy

In [3]:
shadow = 'shadow10_2'
AngleGammma = 'AngleGamma0.400000'
evaluation_path = "../../get_data/recover_test2/" + shadow
protocol = "jbr"
sigo_file_list = []
# sigo_files = glob.glob("../../ns3.30/data/sigo/*")
# sigo_files = glob.glob(evaluation_path + "/sigo/*")
sigo_files = glob.glob(evaluation_path + "/"+protocol+"/*")
for file in sigo_files:
    sigo_file_list.append(file)
    
send_sigo_file_list = []
# send_sigo_files = glob.glob("../../ns3.30/data/send_sigo/*")
# send_sigo_files = glob.glob(evaluation_path + "/send_sigo/*")
send_sigo_files = glob.glob(evaluation_path + "/send_" + protocol + "/*")
for file in send_sigo_files:
    send_sigo_file_list.append(file)
print('receive log')
print(sigo_file_list)
print('send log')
print(send_sigo_file_list)

receive log
['../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10001nodenum_200.csv', '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10001nodenum_300.csv', '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10001nodenum_400.csv', '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10002nodenum_200.csv', '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10002nodenum_300.csv', '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10002nodenum_400.csv', '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10003nodenum_200.csv', '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10003nodenum_300.csv', '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10003nodenum_400.csv', '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10004nodenum_200.csv', '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10004nodenum_300.csv', '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10004nodenum_400.csv', '../../get_data/recover_test2/shadow10_

In [5]:
seed_list = []
nodenum_list = ['200', '300', '400'] ###############nodenumの形式を変更したらここはベタ打ちで変更を加える
regex = re.compile('\d+')
seed_index = 3
node_index = 4

for path in sigo_file_list:
    for line in path.splitlines():
        match = regex.findall(line)
        if match[seed_index] not in seed_list:
            seed_list.append(match[seed_index])
            
print(seed_list)
print('simulation 試行回数',len(seed_list))

['10001', '10002', '10003', '10004', '10005', '10006', '10007', '10008', '10009']
simulation 試行回数 9


In [6]:

sigo_dict = {}
check_seed = seed_list[0]
sigo_dict[check_seed] = {}
for path in sigo_file_list:
    for line in path.splitlines():
      match = regex.findall(line) # match[2] = seed ,match[3] = nudenum
     
    if match[seed_index] == check_seed:
        sigo_dict[match[seed_index]][match[node_index]] = path
    else:#ロープ中seed値が変わるとき
        sigo_dict[match[seed_index]] = {}
        sigo_dict[match[seed_index]][match[node_index]] = path
        check_seed = match[seed_index] #seed値の更新
        
send_sigo_dict = {}
check_seed = seed_list[0]
send_sigo_dict[check_seed] = {}
for path in send_sigo_file_list:
    for line in path.splitlines():
      match = regex.findall(line) # match[2] = seed ,match[3] = nudenum
     
    if match[seed_index] == check_seed:
        send_sigo_dict[match[seed_index]][match[node_index]] = path
    else:#ロープ中seed値が変わるとき
        send_sigo_dict[match[seed_index]] = {}
        send_sigo_dict[match[seed_index]][match[node_index]] = path
        check_seed = match[seed_index] #seed値の更新
        

print(sigo_dict)
print(send_sigo_dict)

{'10001': {'200': '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10001nodenum_200.csv', '300': '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10001nodenum_300.csv', '400': '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10001nodenum_400.csv'}, '10002': {'200': '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10002nodenum_200.csv', '300': '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10002nodenum_300.csv', '400': '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10002nodenum_400.csv'}, '10003': {'200': '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10003nodenum_200.csv', '300': '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10003nodenum_300.csv', '400': '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10003nodenum_400.csv'}, '10004': {'200': '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10004nodenum_200.csv', '300': '../../get_data/recover_test2/shadow10_2/jbr\\sigo-seed_10004nodenum_300.csv', '400': '..

In [7]:
def packet_ratio(path):
    #csvファイルの取得
    path_df = pd.read_csv(path)
    dest_list = path_df['destination_id'].value_counts().index.tolist() #重複なしのdestination_idを格納
    #目的地に届いたidだけをlistに保存
    reach_list = path_df[path_df['recv_id'] == path_df['destination_id']]['destination_id'].value_counts().index.tolist()
#     PDR = len(reach_list)/len(dest_list)
    PDR = len(reach_list)/10
    
#     *************** len(dest_list): destination_idのリストをいったん10個で定義
    return PDR

def packet_hop(path):
    path_df = pd.read_csv(path)
    dest_list = path_df['destination_id'].value_counts().index.tolist() #重複なしのdestination_idを格納
    #目的地に届いたidだけをlistに保存
    reach_list = path_df[path_df['recv_id'] == path_df['destination_id']]['destination_id'] 
    #reach_listを回す
    total_hop = 0
    for reach_id in reach_list:
        hop = path_df[(path_df['destination_id'] == reach_id) & (path_df['destination_id'] == path_df['recv_id'])]['hopcount']
        hop = int(hop.head(1).iloc[-1])
        total_hop += hop
    if(len(reach_list) != 0):
        average_hop = total_hop / len(reach_list)
    else:
        average_hop = total_hop
    
    return average_hop

def packet_delay(path):
    delay_list = []
    #csvファイルの取得
    path_df = pd.read_csv(path)
    #目的地に届いたidだけをlistに保存
    reach_list = path_df[path_df['recv_id'] == path_df['destination_id']]['destination_id'].value_counts().index.tolist()
    #reach_listを回す
    if (len(reach_list) == 0):
        delay = 0
        print('reach_list none')
        return delay
    for reach_id in reach_list:
        source_time = path_df[path_df['destination_id'] == reach_id]['time']
        recv_time = path_df[(path_df['destination_id'] == reach_id) & (path_df['destination_id'] == path_df['recv_id'])]['time']
        delay = int(recv_time.head(1).iloc[-1]) - int(source_time.head(1).iloc[-1])
        delay = delay / 1000000
        delay_list.append(delay)
    delay_mean = np.mean(delay_list)
    return delay_mean

def packet_over_head_send(send_path, recv_path):
    send_path_df = pd.read_csv(send_path)
    send_column_num = len(send_path_df.columns) #カラム数を取得
#     display(path_df.iloc[:,send_column_num-1]) 末尾の取得方法　path_df = pd.read_csv(recv_path)末尾=送信ログ(送信したかどうか)
    #目的地に届いたidだけをlistに保存
    recv_path_df = pd.read_csv(recv_path)
    reach_list = recv_path_df[recv_path_df['recv_id'] == recv_path_df['destination_id']]['destination_id'].value_counts().index.tolist()
    send_count = send_path_df[send_path_df.iloc[:,send_column_num-1] == 1].value_counts() #最後のカラムの値が1
#     print(len(send_count))
    if(len(reach_list) != 0):
        overhead = len(send_count)/len(reach_list)
    else:
        overhead = len(send_count)
    return overhead

In [11]:
def col_average(seeds):
    new_list = []
    for seed in seeds:
        del seed[len(nodenum_list)] #1列ずつ平均を算出するうえで邪魔なseed値を消していく seedの個数 
        new_list.append(seed)
#     np.set_printoptions(precision=3, suppress=True)
    np_new_list = np.array(new_list)
    return np.mean(np_new_list, axis=0)

In [12]:
def header(protocol, writer, index):
    writer.writerow([])
    writer.writerow([protocol, index])
    writer.writerow(['200','300','400','seed'])

In [13]:
def write(seeds):
    for seed in seeds:
        writer.writerow(seed)

with open(evaluation_path + '/' + protocol + '/jbr_test.csv', 'w', newline='') as csv_file:
    #---------------------------PDR---------------------------
    sigo_pdr_average_list = []
    sigo_delay_average_list = []
    sigo_overhead_average_list = []
    sigo_hop_average_list = []
    
    #headerを設定
    writer = csv.writer(csv_file)
    header(protocol, writer, 'PDR')
    #--sigo--#
    for seed in seed_list:
        sigo_pdr_ratio_list = []
        for node in nodenum_list:
            path = sigo_dict[seed][node]
            ratio = packet_ratio(path)
            sigo_pdr_ratio_list.append(ratio)
        sigo_pdr_ratio_list.append(seed)
        writer.writerow(sigo_pdr_ratio_list)
        sigo_pdr_average_list.append(sigo_pdr_ratio_list)
    #------------------------delay-----------------------
    header(protocol, writer, 'delay')
    #--sigo--#
    for seed in seed_list:
        sigo_delay_list = []
        for node in nodenum_list:
            path = sigo_dict[seed][node]
            delay = packet_delay(path)
            sigo_delay_list.append(delay)
        sigo_delay_list.append(seed)
        writer.writerow(sigo_delay_list)
        sigo_delay_average_list.append(sigo_delay_list)    
    #------------------------overhead-----------------------
    header(protocol, writer, 'Overhead')
    #--sigo--#
    for seed in seed_list:
        sigo_overhead_list = []
        for node in nodenum_list:
            recv_path = sigo_dict[seed][node]
            print(seed)
            send_path = send_sigo_dict[seed][node]
#             print('path', 'seed', seed)
            overhead = packet_over_head_send(send_path, recv_path)
            sigo_overhead_list.append(overhead)
        sigo_overhead_list.append(seed)
        writer.writerow(sigo_overhead_list)
        sigo_overhead_average_list.append(sigo_overhead_list)
    #------------------------Hop-----------------------
    header(protocol, writer, 'Hop')
    #--sigo--#
    for seed in seed_list:
        sigo_hop_list = []
        for node in nodenum_list:
            path = sigo_dict[seed][node]
            hop = packet_hop(path)
            sigo_hop_list.append(hop)
        sigo_hop_list.append(seed)
        writer.writerow(sigo_hop_list)
        sigo_hop_average_list.append(sigo_hop_list)
        
    header(protocol, writer , 'PDR-average')
    writer.writerow(col_average(sigo_pdr_average_list))
    header(protocol, writer , 'delay-average')
    writer.writerow(col_average(sigo_delay_average_list))
    header(protocol, writer , 'overhead-average')
    writer.writerow(col_average(sigo_overhead_average_list))
    header(protocol, writer , 'hop-average')
    writer.writerow(col_average(sigo_hop_average_list))
    print('finish')

reach_list none
10001
10001
10001
10002
10002
10002
10003
10003
10003
10004
10004
10004
10005
10005
10005
10006
10006
10006
10007
10007
10007
10008
10008
10008
10009
10009
10009
finish


In [19]:
#sigo
print('sigo -----------------')
for seed in seed_list:
        sigo_delay_list = []
        for node in nodenum_list:
            if(node not in sigo_dict[seed]):
                print("seed", seed, 'node_num', node)

sigo -----------------
seed 10050 node_num 200
seed 10050 node_num 300
seed 10050 node_num 400


In [98]:
evaluation_path + 'Performance_evaluation_5_30.csv'

'../../get_data/interRange_20/shadow20_2Performance_evaluation_5_30.csv'